In [52]:
import pandas as pd
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
from bokeh.palettes import Category20
from bokeh.transform import factor_cmap
from bokeh.models import Legend
from bokeh.models import LegendItem

# Load data
df = pd.read_csv("Air_Traffic_Passenger_Statistics_20240408.csv")
df['Activity Period Start Date'] = pd.to_datetime(df['Activity Period Start Date'])

# Create an empty figure
p = figure(title="Passenger Counts Across Months", x_axis_label="Months", y_axis_label="Passenger Count",
           x_range=(1, 12), width=900)

legend_it = []

# Loop through years
for year in range(2000, 2024):
    # Filter data for the current year
    df_filtered = df[df['Activity Period Start Date'].dt.year == year]
    
    # Group the data by month and sum the passenger counts
    df_grouped = df_filtered.groupby(df_filtered['Activity Period Start Date'].dt.month)['Passenger Count'].sum()
    
    # Create a ColumnDataSource for the current year
    source = ColumnDataSource(data={'Months': df_grouped.index.values,
                                     f'Passenger Count ({year})': df_grouped.values})
    
    # Plot line for the current year using a distinct color from the Category20 palette
    color_index = (year - 2000) % 20
    color = Category20[20][color_index]
    p.line(x='Months', y=f'Passenger Count ({year})', source=source,
           line_width=2, line_color=color)
    
    legend_it.append(LegendItem(label=f'Passenger Count ({year})', renderers=[p.renderers[(year - 2000)]], index=0))

# Customize x-axis ticks
p.xaxis.ticker = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
p.xaxis.major_label_overrides = {1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun',
                                  7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'}

legend = Legend(items=legend_it)
legend.click_policy="mute"

p.add_layout(legend, 'right')

# Show the plot
show(p)
